In [109]:
import sys
import pandas
sys.path.append('../')

from src.utils.graph_utils import partition_graph_by_lonlat
import networkx as nx
from jurbey.jurbey import JURBEY

with open("../data/1558537930325.jurbey", 'rb') as tempf:
    g = JURBEY.load(tempf.read())
print(g.number_of_nodes())
g_partition = partition_graph_by_lonlat(g)


290100


**Convert to edge-based graph**

In [110]:
import networkx as nx
L = nx.line_graph(nx.DiGraph(g_partition))

In [3]:
nodes = list(L.nodes())
g_partition[nodes[10][0]][nodes[10][1]]['data']

Arc(arcType=<ArcType.LANE_STRAIGHT: 'LANE_STRAIGHT'>, roadClass=<RoadClass.MajorRoad: 2>, roadAccessibility=<RoadAccessibility.NoRestriction: 1>, metadata={'bicycle': 'no', 'highway': 'primary', 'lanes': '4', 'lit': 'yes', 'maxspeed': '50', 'name': 'Bismarckstraße', 'oneway': 'yes', 'postal_code': '10625', 'ref': 'B 2;B 5', 'surface': 'asphalt', 'turn:lanes': 'through|through|through;right|right'}, signs=[], vehicleAccessibility=[], geometry=[GeoCoordinates(lon=13.3207077, lat=52.5123944, alt=nan), GeoCoordinates(lon=13.3207877, lat=52.5123711, alt=nan)])

**Extract dynamic (speed) + static features from nodes**

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
enc = OneHotEncoder(handle_unknown='ignore')
ienc = OrdinalEncoder()
scaler = StandardScaler()
def arc_features(arc):
    arc = g_partition[arc[0]][arc[1]]
    return [
        arc['data'].metadata['highway'],
        arc['data'].metadata.get('surface', 'no_sur'),
        arc['data'].roadClass.name
    ],  [float(arc['data'].metadata.get('maxspeed', '50')), 
        int(arc['data'].metadata.get('lanes', '1'))]
def construct_features():
    data = list()
    data_ord = list()
    for node in L.nodes:
        data.append(arc_features(node)[0])
        data_ord.append(arc_features(node)[1])
    return enc.fit_transform(data), ienc.fit_transform(data_ord)
    
x, y = construct_features()
  

In [5]:
enc.categories_

[array(['access_ramp', 'corridor', 'living_street', 'platform', 'primary',
        'residential', 'secondary', 'secondary_link', 'service',
        'tertiary', 'tertiary_link', 'unclassified'], dtype=object),
 array(['asphalt', 'cobblestone', 'cobblestone:flattened', 'concrete',
        'concrete:plates', 'grass_paver', 'no_sur', 'paved',
        'paving_stones', 'sett'], dtype=object),
 array(['DirtRoad', 'LocalRoad', 'MajorRoad'], dtype=object)]

In [6]:
ienc.categories_

[array([ 5., 10., 20., 30., 50.]), array([1., 2., 3., 4., 5.])]

In [7]:
x.shape

(6163, 25)

In [8]:
x

<6163x25 sparse matrix of type '<class 'numpy.float64'>'
	with 18489 stored elements in Compressed Sparse Row format>

**Preprocess adjacency matrix**

In [157]:
adj = nx.to_scipy_sparse_matrix(L, format="coo")
import scipy.sparse as sp
import numpy as np
import torch

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
                                    
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

# build symmetric adjacency matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
                                    
adj = normalize(adj + sp.eye(adj.shape[0]))
adj = sparse_mx_to_torch_sparse_tensor(adj)
                                    

In [10]:
# adj = nx.to_scipy_sparse_matrix(L, format="coo")
# adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [11]:
# def diagonal_degree_matrix(adj):
#     diag = np.zeros([adj.shape[0], adj.shape[0]]) # basically dimensions of your graph
#     rows, cols = adj.nonzero()
#     for row, col in zip(rows, cols):
#         diag[row, row] += 1
#     return diag

# D = diagonal_degree_matrix(adj)
# np.power(D, -0.5).dot(adj).dot(np.power(D, -0.5))

In [12]:
#Our speed data uses segment ids, but the model uses sequential indexes, based on `.nodes()`
import math
id_to_idx = {}
# defaultdict won't do what you expect in Pandas
df = pandas.read_csv("../data/timeseries_speed_april_first_week.csv")
df = df.T
l = (df.isnull().mean() < 0.5).tolist()

indices = [i for i, x in enumerate(l) if x == True]
print(indices)

[4, 7, 16, 124, 157, 204, 351, 352, 379, 389, 430, 487, 492, 513, 539, 551, 568, 569, 587, 613, 630, 639, 649, 675, 680, 698, 711, 713, 780, 782, 784, 865, 1037, 1062, 1070, 1079, 1080, 1088, 1091, 1125, 1206, 1215, 1217, 1238, 1241, 1397, 1400, 1450, 1517, 1550, 1556, 1560, 1564, 1574, 1703, 1733, 1756, 1804, 1805, 1806, 1807, 1819, 1824, 1853, 1907, 1917, 1964, 1996, 1999, 2003, 2007, 2029, 2030, 2033, 2036, 2037, 2040, 2042, 2098, 2099, 2102, 2104, 2116, 2119, 2123, 2128, 2137, 2143, 2210, 2261, 2281, 2284, 2289, 2358, 2382, 2387, 2394, 2406, 2411, 2412, 2473, 2487, 2550, 2568, 2604, 2605, 2649, 2657, 2674, 2677, 2738, 2753, 2766, 2788, 2794, 2809, 2851, 2853, 2869, 2871, 2917, 2925, 2956, 3081, 3086, 3087, 3088, 3090, 3117, 3120, 3132, 3223, 3264, 3450, 3458, 3459, 3530, 3577, 3587, 3627, 3629, 3645, 3728, 3792, 3870, 3974, 3984, 3993, 4006, 4064, 4065, 4083, 4084, 4303, 4445, 4448, 4550, 4551, 4572, 4602, 4655, 4673, 4688, 4698, 4700, 4704, 4798, 4814, 4866, 4889, 4953, 4967, 4992

In [13]:
TOTAL_T_STEPS = 144
id_to_idx = {}

for idx, id_ in enumerate(L.nodes()):
    id_to_idx[id_] = idx
df = df.T
df = df.loc[:, df.columns != 'Unnamed: 0']

df2 = df['from_node']
df3 = df['to_node']

df_filled = df.loc[:, df.columns != 'from_node']
df_filled = df.loc[:, df.columns != 'to_node']


#df_filled = df_filled.interpolate(method='nearest', axis=1)
SPEED_COLUMNS = list(map(str, range(TOTAL_T_STEPS)))
df_filled = df_filled.fillna(df_filled[SPEED_COLUMNS].mean())
df_filled['from_node'] = df2
df_filled['to_node'] = df3

print(df_filled[0:10])


           0         1        2          3          4          5          6  \
0  10.085606  8.881553  7.69121  10.854457   7.423597   5.680978   7.036838   
1  10.085606  8.881553  7.69121  10.854457   7.423597   5.680978   7.036838   
2  10.085606  8.881553  7.69121   7.977541   7.423597  10.486210   5.523238   
3  10.085606  8.881553  7.69121   7.977541   7.423597   6.141520   5.523238   
4  10.085606  8.881553  7.69121  10.468811  10.636621  10.288534  10.617513   
5  10.085606  8.881553  7.69121   7.977541   7.423597   6.141520  10.286999   
6  10.085606  8.881553  7.69121  10.854457   7.423597   5.680978   7.036838   
7  10.278846  8.881553  7.69121   7.977541  10.831409   6.141520   3.140737   
8  10.085606  8.881553  7.69121   7.977541   7.423597  10.298252   1.220956   
9  10.085606  8.881553  7.69121   7.977541   7.423597   6.141520   5.523238   

           7          8          9  ...        136        137        138  \
0   1.499997  10.088634  10.587459  ...  10.581123   7

In [14]:
df_filled[0:10]

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,from_node,to_node
0,10.085606,8.881553,7.69121,10.854457,7.423597,5.680978,7.036838,1.499997,10.088634,10.587459,...,10.581123,7.129785,7.111766,7.056623,7.928614,6.816858,10.850843,10.036408,628154368,1023689595
1,10.085606,8.881553,7.69121,10.854457,7.423597,5.680978,7.036838,1.499997,10.088634,10.587459,...,10.581123,7.129785,7.111766,7.056623,7.928614,6.816858,10.850843,10.036408,628154368,1023689595
2,10.085606,8.881553,7.69121,7.977541,7.423597,10.486210,5.523238,4.824137,10.937218,5.788495,...,7.943949,7.129785,7.111766,7.056623,7.928614,6.816858,7.795600,7.893893,527147009,27537239
3,10.085606,8.881553,7.69121,7.977541,7.423597,6.141520,5.523238,4.824137,6.148046,5.788495,...,7.943949,7.129785,7.111766,7.056623,7.928614,6.816858,7.795600,7.893893,527147009,26908815
4,10.085606,8.881553,7.69121,10.468811,10.636621,10.288534,10.617513,1.758539,3.404401,10.224568,...,6.098767,10.257657,10.586166,7.056623,10.669476,4.313417,10.392901,10.518310,628154370,3804638178
5,10.085606,8.881553,7.69121,7.977541,7.423597,6.141520,10.286999,4.824137,6.148046,10.833570,...,10.537584,10.286999,7.111766,7.056623,7.928614,7.238553,10.497979,10.491733,628154372,26938222
6,10.085606,8.881553,7.69121,10.854457,7.423597,5.680978,7.036838,1.499997,5.044317,10.587459,...,0.961866,7.129785,7.111766,7.056623,7.928614,6.816858,10.850843,10.036408,628154375,1560866145
7,10.278846,8.881553,7.69121,7.977541,10.831409,6.141520,3.140737,2.073876,0.385263,4.104480,...,10.505827,10.517314,7.111766,7.056623,7.928614,10.189923,10.436750,4.196431,5791596551,1321327852
8,10.085606,8.881553,7.69121,7.977541,7.423597,10.298252,1.220956,10.298252,6.148046,5.788495,...,7.943949,7.129785,7.111766,7.056623,7.928614,6.816858,7.795600,7.893893,5791621141,4782446443
9,10.085606,8.881553,7.69121,7.977541,7.423597,6.141520,5.523238,4.824137,6.148046,5.788495,...,7.943949,7.129785,7.111766,7.056623,7.928614,6.816858,7.795600,7.893893,5791621141,26875019


**Create rolling window tensor dataset**

In [15]:
import torch
import scipy.sparse
TOTAL_T_STEPS = 144

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler
enc = OneHotEncoder(handle_unknown='ignore')
ienc = OrdinalEncoder()
scaler = RobustScaler()
import math
  
speed_features = df_filled.values.flatten()
speed_features = np.array([s for s in speed_features if not math.isnan(s)]).reshape(-1,1)
print(speed_features[0:10])  
scaler.fit(speed_features)

[[10.08560558]
 [ 8.88155287]
 [ 7.69121021]
 [10.8544569 ]
 [ 7.42359714]
 [ 5.68097802]
 [ 7.03683772]
 [ 1.49999732]
 [10.08863449]
 [10.58745861]]


RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)

In [16]:
scaler.transform(np.array(speed_features[0:10]).reshape(-1,1))

array([[ 1.81528429],
       [ 1.12561747],
       [ 0.44380359],
       [ 2.25567303],
       [ 0.29051806],
       [-0.70763309],
       [ 0.06898694],
       [-3.10244826],
       [ 1.81701922],
       [ 2.10273964]])

In [17]:
def build_dataset_to_numpy_tensor(from_=0, to=TOTAL_T_STEPS, df=None, norm=False):
    """
    We extract features from speed (actual speed, whether speed is missing)
    and combine with static features.
    :return:
         np.ndarray: dataset tensor of shape [num_time_steps, num_nodes, num_features]
    """
    dataset = list()
    for t in range(from_, to):
        cat_features_at_t = [['primary', 'asphalt', 'MajorRoad']] * len(L.nodes)
        ord_features_at_t = [[50.0, 4]] * len(L.nodes)
        speed_features_at_t = [50] * len(L.nodes) 
        speed_is_nan_feature = [1] * len(L.nodes)
        for _, row in df.iterrows():

            arc = (row['from_node'], row['to_node'])
            cat_features_at_t[id_to_idx[arc]], ord_features_at_t[id_to_idx[arc]]  = arc_features(arc)
            speed_features_at_t[id_to_idx[arc]] = row[str(t)]
            if np.isnan(row[str(t)]): 
                speed_is_nan_feature[id_to_idx[arc]] = 0
        dataset.append(np.concatenate([scaler.transform(np.array(speed_features_at_t).reshape(-1, 1)) if norm else  np.array(speed_features_at_t).reshape(-1, 1), 
                                       np.array(speed_is_nan_feature).reshape(-1, 1), 
                                       ienc.fit_transform(ord_features_at_t),
                                       enc.fit_transform(cat_features_at_t).toarray()], axis=1))
    return np.stack(dataset, axis=0)

Y = build_dataset_to_numpy_tensor(df=df)
Y_filled = build_dataset_to_numpy_tensor(df=df_filled, norm=True)

In [18]:
X = np.moveaxis(Y, source=(0,1,2), destination=(2,0,1))
X_filled = np.moveaxis(Y_filled, source=(0,1,2), destination=(2,0,1))


# num_vertices, num_features, num_timesteps
X.shape

(6163, 29, 144)

In [19]:
# Build mask tensor
X_masked = torch.where(torch.isnan(torch.from_numpy(X)), torch.tensor([0]), torch.tensor([1]))
X_masked = X_masked.bool()
print(X.shape)
print(X_masked.shape)

(6163, 29, 144)
torch.Size([6163, 29, 144])


In [20]:
def generate_dataset_concat(X, X_masked, num_timesteps_input, num_timesteps_output):
    """
    Takes node features for the graph and divides them into multiple samples
    along the time-axis by sliding a window of size (num_timesteps_input+
    num_timesteps_output) across it in steps of 1.
    :param X: Node features of shape (num_vertices, num_features,
    num_timesteps)
    :return:
        - Node data (features + labels) divided into multiple samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_input).
        
    """
    # Generate the beginning index and the ending index of a sample, which
    # contains (num_points_for_training + num_points_for_predicting) points
    indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
               in range(X.shape[2] - (
                num_timesteps_input + num_timesteps_output) + 1)]

    # Save samples
    features, target = [], []
    mask = []
    for i, j in indices:
        features.append(X[:, :, i: i + num_timesteps_input])
        target.append(X[:, 0, i + num_timesteps_input: j])
        mask.append(X_masked[:, 0, i + num_timesteps_input: j])

    return torch.from_numpy(np.array(features)), torch.from_numpy(np.array(target)),torch.stack(mask)

In [21]:
# training, validation, testing : 0.7, 0.1, 0.2
split_line1 = int(X.shape[2] * 0.7)
split_line2 = int(X.shape[2] * 0.9)
train_original_data = X_filled[:, :, :split_line1]
val_original_data = X_filled[:, :, split_line1:split_line2]
test_original_data = X_filled[:, :, split_line2:]

train_mask = X_masked[:, :, :split_line1]
valid_mask = X_masked[:, :, split_line1:split_line2]
test_mask = X_masked[:, :, split_line2:]


look_back = 9
look_ahead = 1
# num_samples, num_nodes, num_timesteps, num_features

training_data, training_target, train_mask = generate_dataset_concat(train_original_data, train_mask,
                                                       num_timesteps_input=look_back,
                                                       num_timesteps_output=look_ahead)
valid_data, valid_target, valid_mask = generate_dataset_concat(val_original_data, valid_mask,
                                             num_timesteps_input=look_back,
                                             num_timesteps_output=look_ahead)
test_data, test_target, test_mask = generate_dataset_concat(test_original_data, test_mask,
                                               num_timesteps_input=look_back,
                                               num_timesteps_output=look_ahead)

print(f"shape of training: {training_data.shape}, {training_target.shape}, {train_mask.shape}")
print(f"shape of validation: {valid_data.shape}, {valid_target.shape}, {valid_mask.shape}")
print(f"shape of testing: {test_data.shape}, {test_target.shape}, {test_mask.shape}")

shape of training: torch.Size([91, 6163, 29, 9]), torch.Size([91, 6163, 1]), torch.Size([91, 6163, 1])
shape of validation: torch.Size([20, 6163, 29, 9]), torch.Size([20, 6163, 1]), torch.Size([20, 6163, 1])
shape of testing: torch.Size([6, 6163, 29, 9]), torch.Size([6, 6163, 1]), torch.Size([6, 6163, 1])


In [22]:
train_mask[1, :, :].shape

torch.Size([6163, 1])

In [23]:
from test_tube import Experiment 
from pytorch_lightning import Trainer
import os

# PyTorch summarywriter with a few bells and whistles    
exp = Experiment(save_dir=os.getcwd())

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=45, train_percent_check=1)

In [24]:
import h5py
with h5py.File('data.hdf5', 'w') as f:
    f.create_dataset("train", data=training_data)
    f.create_dataset("valid", data=valid_data)
    f.create_dataset("test", data=test_data)
with h5py.File('target.hdf5', 'w') as f:
    f.create_dataset("train", data=training_target)
    f.create_dataset("valid", data=valid_target)
    f.create_dataset("test", data=test_target)
with h5py.File('mask.hdf5', 'w') as f:
    f.create_dataset("train", data=train_mask)
    f.create_dataset("valid", data=valid_mask)
    f.create_dataset("test", data=test_mask)

In [25]:
type(test_data)

torch.Tensor

In [26]:
data = {}
data['train'] = training_data
data['valid'] = valid_data
data['test'] = test_data

target = {}
target['train'] = training_target
target['valid'] = valid_target
target['test'] = test_target

mask = {}
mask['train'] = train_mask
mask['valid'] = valid_mask
mask['test'] = test_mask
# batch shape: torch.Size([1, 6163, 26, 10])
print(valid_data.shape)
print(valid_mask[10, :, :])
print(valid_target[10, :, :])

# IMPORTANT: we normalize speed
print(valid_target[10, :, :].masked_select(valid_mask[10, :, :]))

torch.Size([20, 6163, 29, 9])
tensor([[ True],
        [ True],
        [False],
        ...,
        [False],
        [False],
        [False]])
tensor([[1.7845],
        [1.7845],
        [1.1043],
        ...,
        [1.1043],
        [1.1043],
        [1.1043]], dtype=torch.float64)
tensor([ 1.7845e+00,  1.7845e+00,  2.0257e+00,  2.0257e+00,  1.7845e+00,
         1.7845e+00,  2.2493e+00,  2.2178e+00,  2.1416e+00,  2.0101e+00,
        -2.8412e+00,  2.2807e+00,  2.2807e+00,  2.0406e+00,  2.0406e+00,
         2.1416e+00,  2.1416e+00,  2.1416e+00,  2.0257e+00,  2.2178e+00,
         2.2619e+00,  1.7970e+00,  1.7970e+00,  2.1756e+00,  2.1756e+00,
         1.7970e+00,  1.7970e+00,  2.3061e+00,  2.3061e+00,  2.0881e+00,
         2.0881e+00,  2.0557e+00,  2.1464e+00,  2.1039e+00, -3.9616e+00,
        -3.1079e+00,  1.9234e+00, -2.8746e+00,  2.2798e+00,  2.0426e+00,
         1.8834e+00,  2.1416e+00,  2.1416e+00,  1.9875e+00,  1.9982e+00,
         1.9875e+00,  2.1416e+00,  2.1416e+00,  1.8548

**Now start training**

In [97]:
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset, ChainDataset

In [46]:
target['valid'].shape

torch.Size([20, 6163, 1])

In [156]:
adj.shape

AttributeError: 'list' object has no attribute 'shape'

In [129]:
a = torch.split(data['valid'], 4000, dim=1)[0].permute(1,0,2,3)
b = torch.split(target['valid'], 4000, dim=1)[0].permute(1,0,2)
datasets = ConcatDataset((TensorDataset(data['train'].permute(1,0,2,3), target['train'].permute(1,0,2)), TensorDataset(a,b)))

In [170]:
from torch.utils.data import Sampler
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors, adj_tensor):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.adj_tensor = adj_tensor

    def __getitem__(self, index):
        return tuple((tensor[index], self.adj_tensor) for tensor in self.tensors)

    def __len__(self):
        return self.tensors[0].size(0)
    
class CustomSampler(Sampler):

    def __init__(self, data_source, cum_indices, shuffle=True):
        super().__init__(data_source)
        self.data_source = data_source
        self.cum_indices = [0] + cum_indices
        self.shuffle = shuffle

    def __iter__(self):
        batch = []
        for prev, curr in zip(self.cum_indices, self.cum_indices[1:]):
            for idx in range(prev, curr):
                batch.append(idx)
            yield batch
            batch = []

    def __len__(self):
        return len(self.data_source)
adjs = [adj, adj1]
dl = DataLoader(datasets, batch_sampler=CustomSampler(datasets, datasets.cumulative_sizes), shuffle=False)
batches = list()
for batch_nb, batch in enumerate(dl):
    x, y = batch

    print(f'batch_nb: {batch_nb}, adj: {adjs[batch_nb].shape}')
    batches.append(CustomTensorDataset(x, y, adj_tensor=adjs[batch_nb]))
dl = DataLoader(ConcatDataset(batches), batch_size=1, shuffle=False)
def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())
for batch_nb, data_batch in enumerate(dl):
    (x, _adj), (y, _) = data_batch
    _adj = to_sparse(_adj.to_dense().squeeze(dim=0))
    print(_adj.shape)




batch_nb: 0, adj: torch.Size([6163, 6163])
batch_nb: 1, adj: torch.Size([4000, 4000])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])
torch.Size([6163, 6163])


KeyboardInterrupt: 

In [114]:
d2 = TensorDataset(a,b, adj1)

In [113]:
from random import sample
l1 = nx.line_graph(nx.DiGraph(g_partition))
l1 = sample(L.nodes(), 4000)
L1 = nx.line_graph(nx.DiGraph(l1))
adj1 = nx.to_scipy_sparse_matrix(L1, format="coo")

# build symmetric adjacency matrix
adj1 = adj1 + adj1.T.multiply(adj1.T > adj1) - adj1.multiply(adj1.T > adj1)
                                    
adj1 = normalize(adj1 + sp.eye(adj1.shape[0]))
adj1 = sparse_mx_to_torch_sparse_tensor(adj1)
print(adj1.shape)

torch.Size([4000, 4000])


In [ ]:
import imp  
import src.models.tgcn.temporal_spatial_model as l
import src.models.tgcn.layers.lstmcell as h
h = imp.reload(h)

l = imp.reload(l)
TGCN = l.TGCN
model = TGCN(input_dim=29, hidden_dim=29, layer_dim=2, output_dim=1, adjs=(adj, adj1), 
             datasets=data, targets= target, mask=mask, scaler=scaler)
trainer.fit(model)

AAA: <class 'torch.Tensor'>, batch_nb: 0
batch_nb: 0, adj: torch.Size([6163, 6163])
BBB: torch.Size([91, 6163, 1]), torch.Size([91, 6163, 29, 9])


  0%|          | 0/5 [00:00<?, ?it/s]

AAA: <class 'torch.Tensor'>, batch_nb: 1
batch_nb: 1, adj: torch.Size([4000, 4000])
BBB: torch.Size([20, 4000, 1]), torch.Size([20, 4000, 29, 9])
          Name        Type  Params
0      gc_lstm  GCLSTMCell    7830
1  gc_lstm.x2h      Linear    3480
2  gc_lstm.h2h      Linear    3480
3           fc      Linear      30
4      dropout     Dropout       0


  0%|          | 0/131 [00:00<00:28,  4.67it/s]

batch length: 2


...

CCCC: torch.Size([6163, 1]), torch.Size([6163, 1])
batch length: 2


 40%|███▉      | 52/131 [00:37<01:00,  1.32it/s, avg_rmse_loss=5.51, avg_smape_loss=55.1, avg_val_mae=3.06, batch_nb=50, epoch=3, loss=877.018, v_nb=1]

CCCC: torch.Size([6163, 1]), torch.Size([6163, 1])
batch length: 2


 40%|████      | 53/131 [00:38<00:59,  1.32it/s, avg_rmse_loss=5.51, avg_smape_loss=55.1, avg_val_mae=3.06, batch_nb=51, epoch=3, loss=882.917, v_nb=1]

CCCC: torch.Size([6163, 1]), torch.Size([6163, 1])
batch length: 2


In [ ]:
import imp  
import src.tgcn.stgcn as l

# PyTorch summarywriter with a few bells and whistles    
exp = Experiment(save_dir="../data")

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=30, train_percent_check=1)

l = imp.reload(l)
STGCN = l.STGCN
model2 = STGCN(adj=adj, datasets=data, targets=target, mask=mask, scaler=scaler)
trainer.fit(model2)

In [ ]:
import imp  
import src.tgcn.layers.lstm as l

# PyTorch summarywriter with a few bells and whistles    
exp = Experiment(save_dir="../data")

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=30, train_percent_check=1)

l = imp.reload(l)
LSTMs = l.LSTMs
model3 = LSTMs(input_dim=9, datasets=data, targets=target, mask=mask)
trainer.fit(model3)